In [219]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.model_selection import GroupKFold,TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [220]:
pd.options.display.max_columns = 50

In [221]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_set.csv')

/tmp/ipykernel_42161/1480996787.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [222]:
for df in train,test:
    df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])
    

In [223]:
train['name'].nunique()

1017

In [224]:
to_shift_features = ('goals_scored','goals_conceded','assists','bonus'
                     ,'saves','own_goals','penalties_missed','penalties_saved',
                     'red_cards','clean_sheets','minutes','yellow_cards','total_points')


normal_features = (
    'kickoff_time','opponent_team','opp_team_name','round','value','was_home','name','position'
)

In [225]:
train.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'season_x', 'name', 'position', 'team_x',
       'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [226]:
all_feats = to_shift_features+normal_features

In [227]:
train

,Unnamed: 0.1,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15 19:00:00+00:00,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13 14:00:00+00:00,15,17,Spurs,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14 15:00:00+00:00,60,9,Liverpool,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13 14:00:00+00:00,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13 14:00:00+00:00,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101483,101483,101483,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,0,16,Southampton,0,0,0,0,38,0,70366,1.0,4.0,0.0,0,-202,22,224,48,True,0,38
101484,101484,101484,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,1,5,Burnley,0,0,0,0,38,0,15999,2.0,1.0,0.0,1,143,396,253,49,False,0,38
101485,101485,101485,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22 15:00:00+00:00,82,9,Leicester,0,0,0,0,38,0,13856,1.0,4.0,0.0,2,455,683,228,59,False,0,38
101486,101486,101486,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,0,19,West Ham,0,0,0,0,38,0,587,1.0,3.0,0.0,0,-2,0,2,45,True,0,38


In [228]:

def apply_feature_engineering(df):
    df = df.copy()
    cv = GroupKFold(n_splits=df['name'].nunique())
    frames = []
    for _,data in cv.split(df,groups=df['name']):
        data = df.iloc[data]
        data.sort_values('kickoff_time')
        for feat in to_shift_features:
            if feat == 'total_points':
                data[feat+'_lag'] = data[feat].shift(1)
                break
            data[feat] = data[feat].shift(1)

        frames.append(data)
    data = pd.concat(frames)
        
    return data

train = apply_feature_engineering(train)
test = apply_feature_engineering(test)


/tmp/ipykernel_42161/3832799346.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].shift(1)
/tmp/ipykernel_42161/3832799346.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat+'_lag'] = data[feat].shift(1)
/tmp/ipykernel_42161/3832799346.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [229]:
target = train['total_points']

In [230]:
le = LabelEncoder()

In [231]:
categorical = ['position','was_home']

In [232]:
for category in categorical:
    le.fit(train[category])
    for df in train,test:
        df[category] = le.transform(df[category])

In [233]:
train

,Unnamed: 0.1,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,total_points_lag
24,24,24,2016-17,Ben Davies,0,NaN,NaN,NaN,0,NaN,0.0,386,3,NaN,NaN,0.0,0.0,2016-08-13 14:00:00+00:00,NaN,6,Everton,NaN,NaN,NaN,NaN,1,NaN,9018,1.0,1.0,0.0,0,0,0,0,50,0,NaN,1,NaN
223,223,223,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,98,0.0,0.0,0.0,0.0,2016-10-29 14:00:00+00:00,0.0,8,Leicester,0.0,0.0,0.0,0.0,10,0.0,11191,1.0,1.0,0.0,0,-721,167,888,49,1,0.0,10,0.0
441,441,441,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,101,0.0,0.0,0.0,0.0,2016-11-06 12:00:00+00:00,0.0,1,Arsenal,0.0,0.0,0.0,0.0,11,0.0,10303,1.0,1.0,0.0,0,-998,124,1122,48,0,0.0,11,0.0
659,659,659,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,118,0.0,0.0,0.0,0.0,2016-11-19 17:30:00+00:00,0.0,20,West Ham,0.0,0.0,0.0,0.0,12,0.0,9163,2.0,3.0,0.0,0,-1168,26,1194,48,1,0.0,12,0.0
877,877,877,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,123,0.0,0.0,0.0,0.0,2016-11-26 17:30:00+00:00,0.0,4,Chelsea,0.0,0.0,0.0,0.0,13,0.0,8724,1.0,2.0,0.0,0,-444,12,456,48,0,0.0,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75859,75859,75859,2020-21,William Osula,1,Sheffield Utd,NaN,NaN,0,NaN,0.0,711,376,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,4,Burnley,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,1,NaN,38,NaN
75986,75986,75986,2020-21,Nile John,4,Spurs,NaN,NaN,0,NaN,0.0,710,373,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,9,Leicester,NaN,NaN,NaN,NaN,38,NaN,0,4.0,2.0,0.0,0,0,0,0,45,0,NaN,38,NaN
101080,101080,101080,2021-22,Charlie Savage,4,Man Utd,NaN,NaN,0,NaN,0.0,737,376,NaN,NaN,0.0,0.0,2022-05-22 15:00:00+00:00,NaN,7,Crystal Palace,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,0,NaN,38,NaN
75740,75740,75740,2020-21,Sylvester Jasper,4,Fulham,NaN,NaN,0,NaN,0.0,707,371,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,14,Newcastle,NaN,NaN,NaN,NaN,38,NaN,0,2.0,0.0,0.0,0,0,0,0,45,1,NaN,38,NaN


In [234]:
train['Unnamed: 0']

24            24
223          223
441          441
659          659
877          877
           ...  
75859      75859
75986      75986
101080    101080
75740      75740
75631      75631
Name: Unnamed: 0, Length: 101488, dtype: int64

In [235]:
name = test['name']

In [236]:
for df in train,test:
    df.set_index('kickoff_time',inplace=True)
    df.drop(columns=['Unnamed: 0','Unnamed: 0.1','name','season_x','opp_team_name',
                     'team_x','ict_index','influence','element','threat','transfers_in','transfers_out','value','creativity',
                     'team_a_score','team_h_score','transfers_balance','opponent_team','bps','fixture','round'],inplace=True)

In [237]:
train

,position,assists,bonus,clean_sheets,goals_conceded,goals_scored,minutes,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,total_points,was_home,yellow_cards,GW,total_points_lag
kickoff_time,,,,,,,,,,,,,,,,,,
2016-08-13 14:00:00+00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9018,0,0,NaN,1,NaN
2016-10-29 14:00:00+00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11191,0,1,0.0,10,0.0
2016-11-06 12:00:00+00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10303,0,0,0.0,11,0.0
2016-11-19 17:30:00+00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9163,0,1,0.0,12,0.0
2016-11-26 17:30:00+00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8724,0,0,0.0,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-23 15:00:00+00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,NaN,38,NaN
2021-05-23 15:00:00+00:00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,38,NaN
2022-05-22 15:00:00+00:00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,38,NaN


In [238]:
from lightgbm import LGBMRegressor

In [239]:
model = LGBMRegressor(random_state=42)

In [240]:
train['total_points']

kickoff_time
2016-08-13 14:00:00+00:00    0
2016-10-29 14:00:00+00:00    0
2016-11-06 12:00:00+00:00    0
2016-11-19 17:30:00+00:00    0
2016-11-26 17:30:00+00:00    0
                            ..
2021-05-23 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    0
2022-05-22 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    1
Name: total_points, Length: 101488, dtype: int64

In [241]:
cv = TimeSeriesSplit(n_splits=10)


metric = [] 
for train_v,test_v in cv.split(train):
    train_set = train.iloc[train_v]
    validation = train.iloc[test_v]
    model.fit(train_set.drop(columns='total_points'),train_set['total_points'])
    predictions = model.predict(validation.drop(columns='total_points'))
    metric.append(mean_squared_error(validation['total_points'],predictions))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 472
[LightGBM] [Info] Number of data points in the train set: 9228, number of used features: 17
[LightGBM] [Info] Start training from score 2.126463
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001901 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 482
[LightGBM] [Info] Number of data points in the train set: 18454, number of used features: 17
[LightGBM] [Info] Start training from score 2.161266
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [242]:
metric

[7.940817055239908,
 7.691441948514973,
 6.544242943749917,
 5.38208517323608,
 4.406868221794011,
 5.879603636735796,
 4.275994636700067,
 3.715937392336975,
 3.00129317936782,
 2.4799163534234454]

In [243]:
import numpy as np 
np.array(metric).mean()

5.1318200541099

In [244]:
train['total_points'].std()

2.646924569263392

In [245]:
model.fit(train.drop(columns='total_points'),train['total_points'])

pred = model.predict(test.drop(columns='total_points'))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 490
[LightGBM] [Info] Number of data points in the train set: 101488, number of used features: 17
[LightGBM] [Info] Start training from score 1.529245


In [255]:
traintest = pd.concat([train,test])

In [256]:
model.fit(traintest.drop(columns='total_points'),traintest['total_points'])



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016740 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 491
[LightGBM] [Info] Number of data points in the train set: 127993, number of used features: 17
[LightGBM] [Info] Start training from score 1.460424


LGBMRegressor(random_state=42)

In [257]:
mean_squared_error(test['total_points'],pred,squared=False)

1.9839185564854784

In [258]:
test['total_points'].std()

2.3552362999655774

In [259]:
model.feature_name_

['position',
 'assists',
 'bonus',
 'clean_sheets',
 'goals_conceded',
 'goals_scored',
 'minutes',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'saves',
 'selected',
 'was_home',
 'yellow_cards',
 'GW',
 'total_points_lag']

In [260]:
len(['position',
            'assists',
            'bonus',
            'clean_sheets',
            'goals_conceded',
            'goals_scored',
            'minutes',
            'own_goals',
            'penalties_missed',
            'penalties_saved',
            'red_cards',
            'saves',
            'was_home',
            'yellow_cards',
            'GW',
            'total_points',
        ])

16

In [261]:
import joblib 
joblib.dump(model,'./deployment/static/model.lgbm')

['./deployment/static/model.lgbm']

In [262]:
# imp = pd.DataFrame({
#     'name':model.feature_name_,
#     'importance':model.feature_importance
    
# })

In [263]:
len(['position',
 'assists',
 'bonus',
 'clean_sheets',
 'goals_conceded',
 'goals_scored',
 'minutes',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'round',
 'saves',
 'selected',
 'was_home',
 'yellow_cards',
 'GW'])

17

In [264]:
len(['position',
            'assists',
            'bonus',
            'clean_sheets',
            'goals_conceded',
            'goals_scored',
            'minutes',
            'own_goals',
            'penalties_missed',
            'penalties_saved',
            'red_cards',
            'saves',
            'was_home',
            'yellow_cards',
            'GW',
            'total_points',
        ])

16